In [1]:
suppressMessages(library(tidyverse))
suppressMessages(library(reshape2))
source('../0_support-files/theme_CRP-MISC.R')



## 3. For cfDNA abundance, are there any differences between moderate and severe MISC?

### prep meta data and deconvolution

In [2]:
ID_VARS = c("PTID","Sample_ID","Diagnosis","timepoint","expGroup_plotting","scaled_cfDNA","severity")

TISS_COLS = c('bcell','colon','dendritic','eosinophil','erythroblast','heart','hsc','kidney','liver','lung','macrophage','megakaryocyte','monocyte','neutrophil','nkcell','pancreas','progenitor','skin','spleen','tcell')
ID_VARS <- c(ID_VARS,"group")


## Read in MISC data
TOO_df = read.csv("../1_sample-data/STable8_cfdna-samples.csv") %>% 
    filter(Diagnosis == "MIS-C" & timepoint == "acute") %>%
    rename(scaled_cfDNA = cfDNA_concentration) %>%
    mutate(Diagnosis = ifelse(Diagnosis != "Control_Non-inflammatory", 
                              paste0("Acute ",Diagnosis," ",severity),
                                     Diagnosis)) %>%
    rename(Sample_ID = cfdna_sample_id) %>% 
    mutate(expGroup_plotting = Diagnosis)  %>%
    mutate(group = ifelse(severity == 2, "Moderate MIS-C",
                            ifelse(severity == 3,"Severe MIS-C","other")))%>%
    select(all_of(c(ID_VARS,TISS_COLS)))



##------------------------------------
# Calculate differences between groups
mod <- TOO_df %>% filter(severity == 2) %>% pull(scaled_cfDNA)
sev <- TOO_df %>% filter(severity == 3) %>% pull(scaled_cfDNA)

t <- wilcox.test(x = mod, y = sev)
t


	Wilcoxon rank sum exact test

data:  mod and sev
W = 102, p-value = 0.1948
alternative hypothesis: true location shift is not equal to 0


### Plot

In [3]:
##------------------------------------
# Plot

expGroupPalette = c('#008176','#0000a7')
names(expGroupPalette) <- c("Moderate MIS-C","Severe MIS-C")


WIDTH = 1.6
HEIGHT = 1.5

YLIM = 15


pdf(file=paste0("plots/SupPanelA_severity.pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

bxplt <- TOO_df %>%
        ggplot() + 
        geom_boxplot(aes(x=group,y=scaled_cfDNA,fill=group),outlier.shape = NA,size = 0.2, color = "black")+ #, color = "black"
        geom_jitter(aes(x=group,y=scaled_cfDNA),width=.2,height=0,size = 0.01)+
#         geom_point(aes(colour = severity,shape = severity,group=plottingName),position = position_jitterdodge(jitter.width=0.1,seed=100,jitter.height=0),size = 0.5,alpha= 0.5)+       # ,pch=21 ,color = "black"
#         annotate("text", x = 1, y = YLIM - 0.01, label = CELLTYPE,size = 2.5,family = "Helvetica")+
        theme_prevail()+
#         scale_color_manual(values=c('1' = "black",
#                                        '2' = "red")) + 
#         scale_shape_manual(values=c('1'= 16,
#                                        '2'= 4))+
        scale_fill_manual(values=expGroupPalette) + 
        theme(panel.grid.minor = element_blank(),
              axis.title = element_blank(),
             title = element_blank()) +
        scale_y_continuous(breaks = c(0, YLIM/3, (YLIM/3)*2 , YLIM))  + 
        coord_cartesian(ylim = c(0,YLIM*1.05))

print(bxplt)

dev.off()

png 
  2

## 6. Given the severe GI symptoms associated with MISC as well as the finding that the GI tract may act as a reservoir for SARS-CoV-2 resulting in Spike protein antigenemia in MISC, I would be interested in more detail regarding markers of GI injury. Were there any differences between COVID19 and MISC?

In [4]:
ID_VARS = c("PTID","Sample_ID","Diagnosis","timepoint","expGroup_plotting","scaled_cfDNA","severity")

TISS_COLS = c('bcell','colon','dendritic','eosinophil','erythroblast','heart','hsc','kidney','liver','lung','macrophage','megakaryocyte','monocyte','neutrophil','nkcell','pancreas','progenitor','skin','spleen','tcell')
ID_VARS <- c(ID_VARS,"group")

CELLTYPE = "colon"

## Read in MISC data
TOO_df = read.csv("../1_sample-data/STable8_cfdna-samples.csv") %>% 
    filter(Diagnosis %in% c("COVID-19","MIS-C") & timepoint == "acute" & severity >1) %>%
    rename(scaled_cfDNA = cfDNA_concentration) %>%
    # mutate(Diagnosis = ifelse(Diagnosis != "Control_Non-inflammatory", 
    #                           paste0("Acute ",Diagnosis," ",severity),
    #                                  Diagnosis)) %>%
    rename(Sample_ID = cfdna_sample_id) %>% 
    mutate(expGroup_plotting = Diagnosis)  %>%
    mutate(group = Diagnosis)%>%
    select(all_of(c(ID_VARS,TISS_COLS))) %>%
    mutate(colon_conc = colon*scaled_cfDNA)

##------------------------------------
# Calculate differences between groups
mod <- TOO_df %>% filter(Diagnosis == "COVID-19") %>% pull(colon)
sev <- TOO_df %>% filter(Diagnosis == "MIS-C") %>% pull(colon)

t <- wilcox.test(x = mod, y = sev)
t

WIDTH = 1.6
HEIGHT = 1.5

YLIM = 0.03

pdf(file=paste0("plots/SupPanelC_",CELLTYPE,".pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

bxplt <- TOO_df %>%
        ggplot() + 
        geom_boxplot(aes(x=Diagnosis,y=colon,fill=Diagnosis),outlier.shape = NA,size = 0.2, color = "black")+ 
        geom_jitter(aes(x=Diagnosis,y=colon),width=.2,height=0,size = 0.01)+
        theme_prevail()+
        theme(title = element_text(size = 7.5))+
        labs(title = paste0("cfdna\n", gsub("\\."," ",CELLTYPE)))+
        theme(panel.grid.minor = element_blank(),
              axis.title = element_blank(),
             title = element_blank()) +
        scale_y_continuous(breaks = c(0, YLIM/3, (YLIM/3)*2 , YLIM))  + 
        coord_cartesian(ylim = c(0,YLIM*1.05))

print(bxplt)

dev.off()

Warning message in wilcox.test.default(x = mod, y = sev):
“cannot compute exact p-value with ties”



	Wilcoxon rank sum test with continuity correction

data:  mod and sev
W = 262, p-value = 0.404
alternative hypothesis: true location shift is not equal to 0


png 
  2

In [5]:
TOO_df %>% pull(Diagnosis) %>% table()

.
COVID-19    MIS-C 
      11       41 